# IBM Applied Data Science Capstone Course by Coursera

## Week 5 Final Report

##### Analysis of Opening a New Shopping Mall in Bali, Indonesia

### 1. Import Libraries

In [2]:
! pip install geocoder
! pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from IPython.display import display_html

print("Libraries imported.")


     |████████████████████████████████| 102kB 7.8MB/s ta 0:00:011
     |████████████████████████████████| 102kB 9.0MB/s ta 0:00:011
Libraries imported.


In [3]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_districts_of_Bali").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
bali_df = pd.DataFrame({"Neighborhood": neighborhoodList})

bali_df

,Neighborhood
0,"Provinces(provinsi, daerah istimewa)"
1,Regencies (kabupaten)
2,Cities (kota)
3,"Districts(kecamatan, distrik, kapanewon, or ke..."
4,Rural or urban villages(desa or kelurahan)
5,"Abang, Karangasem"
6,"Abiansemal, Badung"
7,"Bangli, Bangli"
8,"Banjar, Buleleng"
9,"Banjarankan, Klungkung"


In [19]:
bali_new_df = bali_df.drop([0,1,2,3,4,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101])

In [20]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Denpasar, Indonesia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [23]:
coords = [ get_latlng(neighborhood) for neighborhood in bali_new_df["Neighborhood"].tolist()]

In [24]:
coords

[[-8.449999999999932, 115.61667000000011],
 [-8.540969999999959, 115.22325000000001],
 [-8.305249999999944, 115.36012000000005],
 [-8.255219999999952, 115.09030000000007],
 [-8.475809999999967, 115.38758000000007],
 [-8.334079999999972, 115.17723000000001],
 [-8.437289999999962, 115.54842000000008],
 [-8.599719999999934, 115.32788000000005],
 [-8.115909999999928, 115.09037000000001],
 [-8.311459999999954, 114.91358000000002],
 [-8.561509999999942, 115.44807000000003],
 [-8.674999999999955, 115.18991000000005],
 [-8.711539999999957, 115.23262000000011],
 [-8.635519999999929, 115.2458200000001],
 [-8.170419999999979, 114.74011000000007],
 [-8.54326999999995, 115.32469000000003],
 [-8.459889999999973, 115.62609000000009],
 [-8.575239999999951, 115.13832000000002],
 [-8.51884999999993, 115.09449000000006],
 [-8.244019999999978, 115.32666000000006],
 [-8.562799999999982, 115.41915000000006],
 [-8.449999999999932, 115.61667000000011],
 [-8.132109999999955, 115.20392000000004],
 [-8.809189999

In [25]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [26]:
bali_new_df['Latitude'] = df_coords['Latitude']
bali_new_df['Longitude'] = df_coords['Longitude']

In [27]:
print(bali_new_df.shape)
bali_new_df

(56, 3)


,Neighborhood,Latitude,Longitude
5,"Abang, Karangasem",-8.334080,115.177230
6,"Abiansemal, Badung",-8.437290,115.548420
7,"Bangli, Bangli",-8.599720,115.327880
8,"Banjar, Buleleng",-8.115910,115.090370
9,"Banjarankan, Klungkung",-8.311460,114.913580
10,"Baturiti, Tabanan",-8.561510,115.448070
11,"Bebandem, Karangasem",-8.675000,115.189910
12,"Blahbatuh, Gianyar",-8.711540,115.232620
13,"Buleleng, Buleleng",-8.635520,115.245820
14,"Busung Biu, Buleleng",-8.170420,114.740110


In [28]:
bali_new_df.to_csv("bali_new_df.csv", index=False)

### Create a map of Denpasar with neighborhoods superimposed on top

In [29]:
address = 'Denpasar, Indonesia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denpasar, Indonesia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denpasar, Indonesia -8.6524973, 115.2191175.


In [33]:
bali_neww_df = bali_new_df.drop([56,57,58,59,60])

In [35]:
map_bali = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(bali_neww_df['Latitude'], bali_neww_df['Longitude'], bali_neww_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bali)  
    
map_bali

In [36]:
map_bali.save('map_bali.html')

### Use Foursquare API to explore the neighborhoods

In [37]:
CLIENT_ID = '4D3RVLMPHNQSMGJQA4XOF4BNCI1NJ5JASEPHGWVNOL10JKLL' # your Foursquare ID
CLIENT_SECRET = 'VKQGAUZFFYMLMPGEZOIRTXV0201HYXUJH1OPIIOL5Q20CHJH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4D3RVLMPHNQSMGJQA4XOF4BNCI1NJ5JASEPHGWVNOL10JKLL
CLIENT_SECRET:VKQGAUZFFYMLMPGEZOIRTXV0201HYXUJH1OPIIOL5Q20CHJH


### Get the top 100 venues that are within a radius of 2000 meters.

In [38]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(bali_neww_df['Latitude'], bali_neww_df['Longitude'], bali_neww_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [39]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(813, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Abang, Karangasem",-8.33408,115.17723,Pia Baturiti Cap Sinar,-8.328171,115.186088,Bakery
1,"Abang, Karangasem",-8.33408,115.17723,Indomaret,-8.326296,115.185904,Food & Drink Shop
2,"Abang, Karangasem",-8.33408,115.17723,Warung Taman Gemitir,-8.337581,115.187883,Café
3,"Abang, Karangasem",-8.33408,115.17723,Ayam Betutu 'Warung Nyoman',-8.337735,115.187611,Fried Chicken Joint
4,"Abang, Karangasem",-8.33408,115.17723,BaliCamp,-8.332449,115.186240,Farm


In [40]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Abang, Karangasem",14,14,14,14,14,14
"Abiansemal, Badung",2,2,2,2,2,2
"Bangli, Bangli",10,10,10,10,10,10
"Banjar, Buleleng",34,34,34,34,34,34
"Banjarankan, Klungkung",1,1,1,1,1,1
"Baturiti, Tabanan",6,6,6,6,6,6
"Bebandem, Karangasem",91,91,91,91,91,91
"Blahbatuh, Gianyar",60,60,60,60,60,60
"Buleleng, Buleleng",60,60,60,60,60,60


### Check total unique categories from all the returned values

In [41]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 164 uniques categories.


In [42]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Food & Drink Shop', 'Café', 'Fried Chicken Joint',
       'Farm', 'Indonesian Restaurant', 'Asian Restaurant', 'Restaurant',
       'Food', 'Arcade', 'Flea Market', 'Campground', 'Diner', 'Mountain',
       'Resort', 'Balinese Restaurant', 'Surf Spot', 'Airport Food Court',
       'Stables', 'Convenience Store', 'Seafood Restaurant', 'Beach',
       'Cosmetics Shop', 'Juice Bar', 'Dim Sum Restaurant', 'Field',
       'Coffee Shop', 'Fast Food Restaurant', 'Electronics Store',
       'Chinese Restaurant', 'Theater', 'Basketball Stadium',
       'Breakfast Spot', 'Supermarket', 'Vegetarian / Vegan Restaurant',
       'Boat or Ferry', 'Pier', 'Movie Theater', 'Hotel', 'Bistro',
       'Clothing Store', 'Javanese Restaurant', 'Bar', 'Noodle House',
       'Soccer Field', 'BBQ Joint', 'Arts & Crafts Store',
       'Indonesian Meatball Place', 'Italian Restaurant',
       'Motorcycle Shop'], dtype=object)

In [44]:
"Neighborhood" in venues_df['VenueCategory'].unique()

False

In [45]:
# one hot encoding
bali_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bali_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bali_onehot.columns[-1]] + list(bali_onehot.columns[:-1])
bali_onehot = bali_onehot[fixed_columns]

print(bali_onehot.shape)
bali_onehot.head()

(813, 165)


,Neighborhoods,Airport,Airport Food Court,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Balinese Restaurant,Bar,Basketball Court,Basketball Stadium,Batik Shop,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Building,Bus Station,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Shop,Dive Spot,Eastern European Restaurant,Electronics Store,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Health Food Store,Historic Site,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Venue,Night Market,Noodle House,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Port,Print Shop,Pub,Public Art,Racetrack,Rafting,Recording Studio,Resort,Rest Area,Restaurant,River,Roof Deck,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Souvenir Shop,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Stables,Steakhouse,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Temple,Thai Restaurant,Theater,Tourist Information Center,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Water Park,Waterfall,Yoga Studio,Zoo,Zoo Exhibit
0,"Abang, Karangasem",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Abang, Karangasem",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Abang, Karangasem",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Abang, Karangasem",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Abang, Karangasem",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
bali_grouped = bali_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bali_grouped.shape)
bali_grouped

(49, 165)


,Neighborhoods,Airport,Airport Food Court,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Balinese Restaurant,Bar,Basketball Court,Basketball Stadium,Batik Shop,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Building,Bus Station,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Shop,Dive Spot,Eastern European Restaurant,Electronics Store,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Health Food Store,Historic Site,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Venue,Night Market,Noodle House,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Port,Print Shop,Pub,Public Art,Racetrack,Rafting,Recording Studio,Resort,Rest Area,Restaurant,River,Roof Deck,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Souvenir Shop,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Stables,Steakhouse,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Temple,Thai Restaurant,Theater,Tourist Information Center,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Water Park,Waterfall,Yoga Studio,Zoo,Zoo Exhibit
0,"Abang, Karangasem",0.00,0.0,0.00,0.000000,0.071429,0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.000000,0.000000,0.00,0.00,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.214286,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.071429,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.000000
1,"Abiansemal, Badung",0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.500

In [47]:
len(bali_grouped[bali_grouped["Shopping Mall"] > 0])

2

### Create a new DataFrame for Shopping Mall only

In [48]:
bali_mall = bali_grouped[["Neighborhoods","Shopping Mall"]]

In [49]:
bali_mall.head()

,Neighborhoods,Shopping Mall
0,"Abang, Karangasem",0.0
1,"Abiansemal, Badung",0.0
2,"Bangli, Bangli",0.0
3,"Banjar, Buleleng",0.0
4,"Banjarankan, Klungkung",0.0


## Cluster Neighborhoods

##### Run k-means to cluster the neighborhoods in Denpasar into 3 clusters.

In [50]:
# set number of clusters
kclusters = 3

bali_clustering = bali_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bali_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [51]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bali_merged = bali_mall.copy()

# add clustering labels
bali_merged["Cluster Labels"] = kmeans.labels_

In [52]:
bali_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bali_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"Abang, Karangasem",0.0,0
1,"Abiansemal, Badung",0.0,0
2,"Bangli, Bangli",0.0,0
3,"Banjar, Buleleng",0.0,0
4,"Banjarankan, Klungkung",0.0,0


In [54]:
bali_merged = bali_merged.join(bali_neww_df.set_index("Neighborhood"), on="Neighborhood")

print(bali_merged.shape)
bali_merged.head() # check the last columns!

(49, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Abang, Karangasem",0.0,0,-8.33408,115.17723
1,"Abiansemal, Badung",0.0,0,-8.43729,115.54842
2,"Bangli, Bangli",0.0,0,-8.59972,115.32788
3,"Banjar, Buleleng",0.0,0,-8.11591,115.09037
4,"Banjarankan, Klungkung",0.0,0,-8.31146,114.91358


In [55]:
# sort the results by Cluster Labels
print(bali_merged.shape)
bali_merged.sort_values(["Cluster Labels"], inplace=True)
bali_merged

(49, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Abang, Karangasem",0.000000,0,-8.334080,115.177230
26,"Marga, Tabanan",0.000000,0,-8.673910,115.552020
27,"Melaya, Jembrana",0.000000,0,-8.396150,115.251140
28,"Mendoyo, Jembrana",0.000000,0,-8.427670,114.839130
29,"Mengwi, Badung",0.000000,0,-8.397310,115.134620
30,"Negara, Jembrana",0.000000,0,-8.356480,115.211370
31,"Nusa Penida, Klungkung",0.000000,0,-8.319200,115.017560
32,"Payangan, Gianyar",0.000000,0,-8.450000,115.616670
33,"Pekutatan, Jembrana",0.000000,0,-8.080170,115.148520
34,"Penebel, Tabanan",0.000000,0,-8.450000,115.616670


##### Visualize resulting clusters

In [56]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bali_merged['Latitude'], bali_merged['Longitude'], bali_merged['Neighborhood'], bali_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### Examine Clusters

In [58]:
bali_merged.loc[bali_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Abang, Karangasem",0.0,0,-8.334080,115.177230
26,"Marga, Tabanan",0.0,0,-8.673910,115.552020
27,"Melaya, Jembrana",0.0,0,-8.396150,115.251140
28,"Mendoyo, Jembrana",0.0,0,-8.427670,114.839130
29,"Mengwi, Badung",0.0,0,-8.397310,115.134620
30,"Negara, Jembrana",0.0,0,-8.356480,115.211370
31,"Nusa Penida, Klungkung",0.0,0,-8.319200,115.017560
32,"Payangan, Gianyar",0.0,0,-8.450000,115.616670
33,"Pekutatan, Jembrana",0.0,0,-8.080170,115.148520
34,"Penebel, Tabanan",0.0,0,-8.450000,115.616670


##### Cluster 1

In [59]:
bali_merged.loc[bali_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
20,"Klungkung, Klungkung",0.05,1,-8.73854,115.18027


##### Cluster 2

In [63]:
bali_merged.loc[bali_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
18,"Kerambitan, Tabanan",0.013699,2,-8.80919,115.15787


##### Cluster 3

In [64]:
bali_merged.loc[bali_merged['Cluster Labels'] == 3]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
